## 1. Tải Dữ Liệu từ CSV

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from datasets import Dataset

def load_csv_dataset(csv_path, text_column, label_column):
    """
    Tải dataset từ file CSV và đổi tên cột.

    Args:
        csv_path (str): Đường dẫn đến file .csv.
        text_column (str): Tên cột chứa văn bản.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        Dataset: Tập dữ liệu đã tải từ file .csv.
    """
    # Tải dữ liệu từ file .csv
    dataset = Dataset.from_csv(csv_path)
    # Đổi tên cột
    dataset = dataset.rename_columns({text_column: "text", label_column: "label"})
    return dataset

# Sử dụng hàm
csv_path = "/content/chatbot_intent_data_v1.csv"             # Đường dẫn file CSV
text_column = "text"       # Cột chứa văn bản
label_column = "intent"        # Cột chứa nhãn

# Tải dataset
dataset = load_csv_dataset(csv_path, text_column, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 28
})
Dataset({
    features: ['text', 'label'],
    num_rows: 10
})


In [8]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["text"], str) or sample["text"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [7]:
def preprocess_text(sample):
    if not isinstance(sample["text"], str) or sample["text"].strip() == "":
        sample["text"] = "EMPTY_TEXT"  # Giá trị mặc định
    return sample

dataset = dataset.map(preprocess_text)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [9]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["text"], str) or sample["text"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)


===== Invalid Samples =====
[]


In [10]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.2)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train = train_dataset.select(range(5))  # Lấy 10 mẫu đầu tiên từ train
sample_test = test_dataset.select(range(2))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train)
print("Sample test dataset:", sample_test)

Chia dataset: 22 mẫu train, 6 mẫu test
Train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 22
})
Test dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 6
})
Sample train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 5
})
Sample test dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 2
})


In [11]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn từ dataset
datasets = [train_dataset, test_dataset]
label_to_int = create_label_mapping(datasets)

# Áp dụng chuyển đổi nhãn
train_dataset = train_dataset.map(preprocess_labels, fn_kwargs={"label_to_int": label_to_int})
test_dataset = test_dataset.map(preprocess_labels, fn_kwargs={"label_to_int": label_to_int})

sample_train = train_dataset.select(range(5))  # Lấy 10 mẫu đầu tiên từ train
sample_test = test_dataset.select(range(2))    # Lấy 10 mẫu đầu tiên từ test


# Kiểm tra dữ liệu sau khi chuyển đổi
print("Mẫu train sau chuyển đổi:", train_dataset.select(range(3)))
print("Mẫu test sau chuyển đổi:", test_dataset.select(range(2)))


print("Sample train dataset:", sample_train)


Ánh xạ nhãn: {'Fallback': 0, 'Im lặng': 1, 'Không chắc chắn': 2, 'Từ chối': 3, 'Đồng ý': 4}


Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Mẫu train sau chuyển đổi: Dataset({
    features: ['text', 'label'],
    num_rows: 3
})
Mẫu test sau chuyển đổi: Dataset({
    features: ['text', 'label'],
    num_rows: 2
})
Sample train dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 5
})


In [12]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 22
})


```
Label Mapping:

Ánh xạ nhãn đã được tự động tạo thành công, các nhãn được ánh xạ vào các số nguyên.
Đây là bước quan trọng để chuyển đổi nhãn chuỗi (text labels) thành số nguyên cho mô hình.
Dataset Mapping:

Dữ liệu huấn luyện và kiểm thử (train_dataset và test_dataset) đã được chuyển đổi thành công với nhãn dạng số.
Số lớp của mô hình:

Model BERTIntentClassification cần được cấu hình lại để số lượng lớp (nhãn) là num_classes=5.
```

## 2. Chuẩn bị tokenizer và token hoá dữ liệu

In [14]:
from transformers import AutoTokenizer

def initialize_model_and_tokenizer(model_class, model_name="bert-base-uncased", num_classes=2, max_seq_length=512, cache_dir="huggingface"):
    """
    Khởi tạo tokenizer, model và thiết lập thông số.

    Args:
        model_class (class): Lớp model cần khởi tạo.
        model_name (str): Tên model pretrained từ Hugging Face.
        num_classes (int): Số lượng nhãn đầu ra.
        max_seq_length (int): Độ dài tối đa của chuỗi token.
        cache_dir (str): Thư mục lưu trữ cache của mô hình.

    Returns:
        tuple: (tokenizer, model, max_seq_length)
    """
    try:
        # Khởi tạo tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
        print(f"Tokenizer '{model_name}' đã được khởi tạo thành công.")

        # Khởi tạo model
        model = model_class(model_name=model_name, num_classes=num_classes)
        print(f"Model '{model_name}' đã được khởi tạo thành công.")

        # Đóng băng các tham số của BERT
        if hasattr(model, "freeze_bert"):
            model.freeze_bert()
            print("Các tham số của BERT đã được đóng băng.")
        else:
            print("Lớp model không có phương thức 'freeze_bert'.")

        return tokenizer, model, max_seq_length
    except Exception as e:
        print(f"Lỗi khi khởi tạo model hoặc tokenizer: {e}")
        raise


# Khởi tạo tokenizer và model
tokenizer, model, max_seq_length = initialize_model_and_tokenizer(
    model_class=BERTIntentClassification,
    model_name="bert-base-uncased",
    num_classes=5,
    max_seq_length=512,
    cache_dir="huggingface"
)

# # Kiểm tra kết quả
# print(tokenizer)
# print(model)
# print(f"Max sequence length: {max_seq_length}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer 'bert-base-uncased' đã được khởi tạo thành công.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model 'bert-base-uncased' đã được khởi tạo thành công.
Các tham số của BERT đã được đóng băng.


Trong đoạn mã của bạn, `tokenizer` là một đối tượng dùng để chuyển đổi văn bản (text) thành các định dạng số mà mô hình BERT có thể hiểu được. Cụ thể, `tokenizer` được khởi tạo bằng cách sử dụng `AutoTokenizer` từ thư viện `transformers`.

---

### **Vai trò của Tokenizer**
1. **Chuyển đổi từ sang token**:
   Tokenizer sẽ chia nhỏ một câu thành các từ hoặc đơn vị nhỏ hơn gọi là *tokens*.
   Ví dụ:
   ```python
   text = "Hello, how are you?"
   tokens = ["hello", ",", "how", "are", "you", "?"]
   ```

2. **Chuyển đổi token thành ID**:
   Tokenizer sẽ ánh xạ từng *token* sang một số nguyên dựa trên từ điển của mô hình (vocabulary).
   Ví dụ:
   ```python
   tokens = ["hello", ",", "how", "are", "you", "?"]
   token_ids = [7592, 1010, 2129, 2024, 2017, 1029]
   ```

3. **Thêm token đặc biệt**:
   Tokenizer sẽ thêm các token đặc biệt mà mô hình yêu cầu, chẳng hạn như `[CLS]` (bắt đầu câu) và `[SEP]` (kết thúc câu).

4. **Tạo Attention Mask**:
   Tokenizer sẽ tạo một `attention_mask` để xác định các token thực sự và các token đệm (padding).

---

### **Token hóa trong đoạn mã**
Khi bạn khởi tạo `tokenizer` bằng `initialize_model_and_tokenizer`, đây là những gì diễn ra:

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir="huggingface")
```

- **`AutoTokenizer.from_pretrained`**:
  - Tải về bộ tokenizer đã được huấn luyện sẵn tương ứng với mô hình `bert-base-uncased`.
  - Bộ tokenizer này đã được cấu hình để làm việc với mô hình BERT.

- **`cache_dir="huggingface"`**:
  - Chỉ định thư mục lưu trữ bộ tokenizer, giúp giảm thời gian tải về trong các lần sử dụng sau.

---

### **Cách sử dụng `tokenizer`**
Bạn có thể sử dụng `tokenizer` để token hóa văn bản đầu vào:

#### **Ví dụ 1: Token hóa một câu**
```python
text = "Hello, how are you?"
encoded = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")

print(encoded)
```
Kết quả:
```python
{
  'input_ids': tensor([[ 101

## Testing 1 câu trước huấn luyện models

In [15]:
print("===== Tokenizer Info =====")
print("Vocab size:", tokenizer.vocab_size)
print("Max model input size:", tokenizer.model_max_length)
print("Special tokens:", tokenizer.special_tokens_map)


===== Tokenizer Info =====
Vocab size: 30522
Max model input size: 512
Special tokens: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


```
 Tokenizer Info
Vocab size (30522): Đây là số lượng từ vựng mà tokenizer của mô hình BERT hỗ trợ. Đây là con số mặc định của bert-base-uncased.
Max model input size (512): Đây là độ dài tối đa của chuỗi đầu vào mà mô hình BERT hỗ trợ.
Special tokens: Các token đặc biệt như [UNK] (unknown), [SEP] (separator), [PAD] (padding), [CLS] (classification), [MASK] (masking) được khởi tạo đúng.
```

In [16]:
print("\n===== Model Info =====")
print("Model architecture:", model.bert.config.architectures)
print("Hidden size:", model.bert.config.hidden_size)
print("Number of classes:", model.ffnn[-1].out_features)  # Lớp đầu ra của FFNN



===== Model Info =====
Model architecture: ['BertForMaskedLM']
Hidden size: 768
Number of classes: 5


In [17]:
print("\n===== Thử Tokenizer với một câu ví dụ=====")
test_sentence = "Hello, how are you?"
encoded = tokenizer(test_sentence, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")

print("\n===== Tokenizer Output =====")
print("Input IDs:", encoded["input_ids"])
print("Attention Mask:", encoded["attention_mask"])



===== Thử Tokenizer với một câu ví dụ=====

===== Tokenizer Output =====
Input IDs: tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


```
Tokenizer Output
Input IDs:

[101, 7592, 1010, 2129, 2024, 2017, 1029, 102]: Đây là danh sách các ID của token đã được mã hóa từ câu "Hello, how are you?".
[101] và [102]: Tương ứng với token [CLS] và [SEP], được tự động thêm vào bởi tokenizer.
Các từ như "Hello" (7592), "how" (2129) được ánh xạ chính xác.
Attention Mask:

[1, 1, 1, 1, 1, 1, 1, 1]: Tất cả các token đều được chú ý (không có padding), phù hợp với độ dài chuỗi ngắn.
```

In [18]:
print("\n===== Forward Pass Output =====")
with torch.no_grad():  # Không cần tính gradient
    input_ids = encoded["input_ids"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    attention_mask = encoded["attention_mask"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    logits = model(input_ids=input_ids, attention_mask=attention_mask)

print("Logits shape:", logits.shape)
print("Logits:", logits)



===== Forward Pass Output =====
Logits shape: torch.Size([1, 5])
Logits: tensor([[-0.7042,  0.3137, -0.5307,  0.5366,  0.4700]])


```
Forward Pass Output
Logits Shape:
torch.Size([1, 2]): Kích thước đầu ra đúng với batch size = 1 và số lớp = 2.
Logits:
tensor([[0.8322, 0.3959]]): Đây là giá trị raw logits cho hai lớp.
Lớp đầu tiên (0.8322) có xác suất cao hơn, vì vậy model có thể dự đoán lớp 0.
```

```
Logits Shape:

torch.Size([1, 5]): Đầu ra của mô hình có kích thước chính xác, với batch size là 1 và số lớp là 5.
Logits:

tensor([[ 0.4547, 0.8370, 0.6263, 0.5780, -0.0866]]): Đây là các giá trị raw logits, đại diện cho sự tự tin của mô hình với từng lớp.
Logits không phải xác suất, nhưng lớp có giá trị cao nhất thường sẽ được dự đoán. Trong trường hợp này, lớp có giá trị 0.8370 (lớp thứ 2) có khả năng cao nhất.

```

In [19]:
# Tính xác suất với softmax
softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(logits)

print("Probabilities:", probabilities)
print("Predicted class:", torch.argmax(probabilities).item())


Probabilities: tensor([[0.0858, 0.2375, 0.1021, 0.2968, 0.2777]])
Predicted class: 3


```
Probabilities:

tensor([[0.1864, 0.2731, 0.2212, 0.2108, 0.1085]]):
Đây là xác suất cho từng lớp, tính bằng softmax.
Tổng xác suất:
0.1864
+
0.2731
+
0.2212
+
0.2108
+
0.1085
=
1
0.1864+0.2731+0.2212+0.2108+0.1085=1, đảm bảo tính chính xác của softmax.
Predicted class:

1: Lớp có xác suất cao nhất (27.31%).
Lớp này tương ứng với nhãn "Im lặng" theo ánh xạ nhãn nếu bạn đã định nghĩa:
python
Copy code
{'Fallback': 0, 'Im lặng': 1, 'Không chắc chắn': 2, 'Từ chối': 3, 'Đồng ý': 4}
```

In [20]:
print("\n===== Compute Loss =====")
labels = torch.tensor([1]).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Nhãn mẫu
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)
print("Loss:", loss.item())



===== Compute Loss =====
Loss: 1.4374569654464722


```
Compute Loss
Loss (0.9349):
Loss được tính toán dựa trên CrossEntropyLoss với nhãn thật labels = 1.
Giá trị này hợp lý và cho thấy rằng model đang hoạt động đúng cách, dù cần cải thiện thêm qua huấn luyện.
```

Kết quả **Loss: 1.2977995872497559** cho thấy mô hình của bạn đã tính toán loss một cách chính xác với `CrossEntropyLoss`.

---

### **Phân tích kết quả**
1. **Loss Function**:
   - `CrossEntropyLoss` so sánh logits (chưa qua softmax) với nhãn thật (`labels`), tự động áp dụng softmax trong quá trình tính toán.
   - Giá trị loss dương (~1.2978) là hợp lý trong giai đoạn đầu hoặc khi mô hình chưa được fine-tuned đầy đủ.

2. **Cách tính**:
   - Logits đầu ra của mô hình:
     ```plaintext
     tensor([[ 0.4547,  0.8370,  0.6263,  0.5780, -0.0866]])
     ```
   - Nhãn thật (`labels`): `1` (ứng với lớp `"Im lặng"`).
   - Loss là kết quả của:
     \[
     \text{Loss} = -\log(\text{probability của nhãn thật})
     \]
     Với xác suất từ logits qua softmax:
     ```plaintext
     tensor([[0.1864, 0.2731, 0.2212, 0.2108, 0.1085]])
     ```
     Loss:
     \[
     \text{Loss} = -\log(0.2731) \approx 1.2978
     \]

### **Giải thích quá trình test với 1 câu trước khi huấn luyện mô hình**

---

#### **Quá trình test 1 câu**
1. **Mục tiêu**:
   - Kiểm tra pipeline từ **tokenizer** đến **forward pass** của mô hình có hoạt động đúng không.
   - Đảm bảo rằng các thành phần chính (tokenizer, model, logits, loss function) được kết nối mạch lạc.

2. **Các bước thực hiện**:
   - **Bước 1**: Tokenize một câu ví dụ (như `"Hello, how are you?"`) để chuyển thành `input_ids` và `attention_mask`.
   - **Bước 2**: Truyền dữ liệu qua mô hình (`forward pass`) để lấy logits (đầu ra raw của mô hình).
   - **Bước 3**: Sử dụng một nhãn giả (ví dụ: `1`) để tính loss bằng `CrossEntropyLoss`.

---

#### **Lý do thực hiện khi model chưa huấn luyện**
- **Đảm bảo pipeline hoạt động**:
  - Xác minh rằng tokenizer hoạt động đúng (tokenize và trả về `input_ids`, `attention_mask`).
  - Kiểm tra rằng mô hình BERT và tầng phân loại FFNN hoạt động, trả về logits với đúng kích thước `[batch_size, num_classes]`.
  - Tính loss đúng mà không gặp lỗi như `shape mismatch`.

- **Kiểm tra kích thước và logic**:
  - Đảm bảo rằng logits có đúng số lượng lớp (`num_classes`).
  - Softmax và CrossEntropyLoss tính toán đúng xác suất và loss.

- **Xây dựng nền tảng**:
  - Việc kiểm tra giúp phát hiện lỗi sớm trước khi huấn luyện trên dữ liệu lớn, tránh mất thời gian xử lý khi pipeline bị lỗi.

---

#### **Kết quả thu được khi model chưa huấn luyện**
1. **Pipeline hoạt động**:
   - Model trả về logits với đúng kích thước, cho thấy nó có thể tiếp nhận dữ liệu và tính toán đầu ra.
   - Loss được tính toán đúng dựa trên logits và nhãn giả.

2. **Hiệu suất mô hình**:
   - Giá trị logits và loss cao, ngẫu nhiên, không có ý nghĩa vì mô hình chưa được huấn luyện.
   - Kết quả này là bình thường khi model chỉ sử dụng trọng số được khởi tạo ngẫu nhiên từ BERT.

---

#### **Tóm lại**
- **Mục đích**: Kiểm tra pipeline từ tokenizer đến model và loss function hoạt động mà không gặp lỗi.
- **Khi nào thực hiện**: Luôn thực hiện trước khi huấn luyện để kiểm tra tính logic của pipeline.
- **Kết quả**:
  - Model trả về logits đúng kích thước.
  - Loss được tính đúng dựa trên nhãn giả.
  - Kết quả loss cao là bình thường vì mô hình chưa học gì.

Bước này giống như một kiểm tra chức năng để xác nhận rằng tất cả các thành phần đều làm việc đúng trước khi tiến hành huấn luyện trên dữ liệu thực tế.

### **Mô phỏng thông luồng với 3 câu input**
Dưới đây là quy trình xử lý pipeline với một batch gồm 3 câu input từ **tokenization** đến **tính loss**.

---

#### **Input mẫu**
```python
sentences = [
    "Hello, how are you?",
    "I am not sure about that.",
    "Please let me know your decision."
]
```

---

### **Code thực hiện**
```python
# 1. Tokenize batch input
encoded_batch = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=max_seq_length,
    return_tensors="pt"
)

print("===== Tokenizer Output =====")
print("Input IDs:", encoded_batch["input_ids"])
print("Attention Mask:", encoded_batch["attention_mask"])

# 2. Forward pass qua mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    input_ids = encoded_batch["input_ids"].to(device)
    attention_mask = encoded_batch["attention_mask"].to(device)
    logits_batch = model(input_ids=input_ids, attention_mask=attention_mask)

print("\n===== Forward Pass Output =====")
print("Logits shape:", logits_batch.shape)  # Expect [batch_size, num_classes]
print("Logits:", logits_batch)

# 3. Gán nhãn giả cho batch
labels_batch = torch.tensor([1, 2, 0]).to(device)  # Nhãn giả cho 3 câu input

# 4. Tính loss
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits_batch, labels_batch)
print("\n===== Compute Loss =====")
print("Loss:", loss.item())
```

---

### **Kết quả mong đợi**

#### **Tokenizer Output**
```plaintext
===== Tokenizer Output =====
Input IDs:
tensor([[  101,  7592,  1010,  2129,  2024,  2017,  1029,   102,     0,     0],
        [  101,  1045,  2572,  2025,  2469,  2055,  2008,  1012,   102,     0],
        [  101,  3531,  2292,  2033,  2113,  2115,  3899,  1012,   102,     0]])
Attention Mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])
```

- **Input IDs**: Mỗi câu được mã hóa thành các ID của token. Các chuỗi được padding để cùng độ dài.
- **Attention Mask**: 1 cho token thật, 0 cho token padding.

---

#### **Forward Pass Output**
```plaintext
===== Forward Pass Output =====
Logits shape: torch.Size([3, 5])
Logits:
tensor([[ 0.6547,  0.5370,  0.9263,  0.3780, -0.2866],
        [ 0.2347,  0.8370,  0.1263,  0.4780, -0.1866],
        [ 0.7547,  0.2370,  0.5263,  0.6780,  0.1866]])
```

- **Logits Shape**: `[batch_size=3, num_classes=5]`.
- **Logits**: Giá trị raw logits cho từng lớp, chưa chuyển thành xác suất.

---

#### **Compute Loss**
```plaintext
===== Compute Loss =====
Loss: 1.5738
```

- **Loss**: Giá trị loss được tính dựa trên nhãn giả `[1, 2, 0]` và logits đầu ra.
- Giá trị loss khá cao, điều này bình thường vì mô hình chưa được huấn luyện.

---

### **Ý nghĩa và Lợi ích**
1. **Thông luồng với batch nhiều câu**:
   - Đảm bảo pipeline hoạt động đúng với batch lớn, mô phỏng quá trình huấn luyện thực tế.
   - Kiểm tra khả năng xử lý padding và attention mask.

2. **Kiểm tra output của mô hình**:
   - Logits có đúng kích thước `[batch_size, num_classes]`.
   - Loss được tính đúng dựa trên batch logits và batch labels.

3. **Kết quả khi model chưa huấn luyện**:
   - Logits và loss ngẫu nhiên, cho thấy model hoạt động nhưng chưa có khả năng phân loại hiệu quả.

---

### **Tiếp theo**
- **Tiến hành huấn luyện mô hình**: Sử dụng pipeline này để truyền dữ liệu thực tế qua `TrainerCustom`.
- **Đánh giá sau huấn luyện**: Kiểm tra hiệu suất mô hình trên tập test với các chỉ số như accuracy và F1 score.

Nếu bạn cần hỗ trợ thêm về huấn luyện hoặc đánh giá, hãy cho mình biết nhé! 😊

## collate_fn:

Hàm `collate_fn` là một hàm được sử dụng để **chuẩn bị dữ liệu đầu vào** cho mô hình trong quá trình huấn luyện hoặc đánh giá. Nó được thiết kế để:

1. **Tạo batch dữ liệu** từ các mẫu riêng lẻ.
2. **Tokenize** văn bản để phù hợp với mô hình BERT.
3. **Chuẩn hóa kích thước** các chuỗi bằng cách thêm padding.
4. Trả về dữ liệu đã chuẩn bị ở dạng tensor PyTorch để mô hình có thể xử lý.

---

### **Phân tích hàm `collate_fn`**

#### **Mục đích**
Hàm này thực hiện các bước cần thiết để chuyển đổi dữ liệu thô (văn bản + nhãn) thành dạng tensor mà mô hình có thể sử dụng.

---

#### **Thành phần chính**
1. **Đầu vào (`features`)**:
   - `features` là danh sách các mẫu dữ liệu.
   - Mỗi mẫu là một dictionary với:
     - `"text"`: Văn bản đầu vào.
     - `"label"`: Nhãn của văn bản (dạng số nguyên).

2. **Xử lý dữ liệu**:
   - Tách danh sách văn bản và nhãn:
     ```python
     inputs = [feature["text"] for feature in features]  # Văn bản
     labels = [feature["label"] for feature in features]  # Nhãn
     ```
   - Nhãn được chuyển thành tensor PyTorch:
     ```python
     labels = torch.tensor(labels, dtype=torch.long)
     ```

3. **Token hóa văn bản**:
   - Sử dụng tokenizer (từ Hugging Face) để:
     - Thêm token đặc biệt `[CLS]` và `[SEP]`.
     - Cắt chuỗi vượt quá `max_seq_length`.
     - Thêm padding để tất cả các chuỗi trong batch có cùng độ dài.
     - Trả về tensor PyTorch:
       ```python
       token_inputs = tokenizer(
           inputs,
           add_special_tokens=True,
           truncation=True,
           padding=True,
           max_length=max_seq_length,
           return_tensors="pt"
       )
       ```

4. **Trả về dữ liệu batch**:
   - Batch dữ liệu gồm:
     - `input_ids`: IDs của các token.
     - `attention_mask`: Mask để biểu thị các vị trí padding.
     - `labels`: Nhãn của các mẫu trong batch.

---

#### **Trả về**
Hàm trả về một dictionary có cấu trúc:
```python
{
    "input_ids": torch.Tensor,        # IDs của token (shape: [batch_size, max_seq_length])
    "attention_mask": torch.Tensor,  # Attention mask (shape: [batch_size, max_seq_length])
    "labels": torch.Tensor           # Nhãn của batch (shape: [batch_size])
}
```

---

### **Ứng dụng của `collate_fn`**
1. **Sử dụng trong DataLoader**:
   - Khi huấn luyện mô hình, `collate_fn` được truyền vào `DataLoader` để chuẩn bị batch dữ liệu:
     ```python
     from torch.utils.data import DataLoader

     train_dataloader = DataLoader(
         dataset=train_dataset,
         batch_size=16,
         collate_fn=collate_fn
     )
     ```

2. **Tích hợp với Hugging Face Trainer**:
   - Hàm này giúp chuẩn bị batch đầu vào khi sử dụng các lớp `Trainer` hoặc `DataLoader` tùy chỉnh.

---

### **Kết luận**
Hàm `collate_fn` là một phần quan trọng trong pipeline huấn luyện hoặc đánh giá mô hình BERT. Nó giúp chuẩn bị dữ liệu một cách nhất quán và hiệu quả, đảm bảo rằng dữ liệu đầu vào phù hợp với yêu cầu của mô hình.

In [21]:
def collate_fn(features):
    """
    Chuẩn bị batch dữ liệu để huấn luyện mô hình BERT.

    Args:
        features (list):
            Danh sách các mẫu dữ liệu đầu vào, mỗi mẫu là một dictionary chứa:
                - "text" (str): Chuỗi văn bản.
                - "label" (int): Nhãn tương ứng với văn bản.

    Returns:
        dict:
            Batch dữ liệu đã được token hóa và chuyển đổi thành tensor PyTorch, gồm các thành phần:
                - "input_ids" (torch.Tensor): Tensor chứa token IDs của batch, đã được padding và truncation.
                - "attention_mask" (torch.Tensor): Tensor mask để biểu thị các vị trí padding.
                - "labels" (torch.Tensor): Tensor chứa nhãn của batch.
    """
    # Kiểm tra đầu vào batch (in một mẫu đầu tiên nếu batch quá lớn)
    print("\n===== Features in Batch (First Sample) =====")
    print(features[0] if len(features) > 0 else "Batch is empty.")

    # Lấy danh sách văn bản và nhãn từ dataset
    inputs = [feature["text"] for feature in features]
    labels = [feature["label"] for feature in features]

    # Kiểm tra kích thước dữ liệu sau khi tách
    print(f"\n===== Batch Size: {len(inputs)} =====")
    print("Sample Text:", inputs[0] if len(inputs) > 0 else "No text available.")
    print("Sample Label:", labels[0] if len(labels) > 0 else "No label available.")

    # Chuyển nhãn thành tensor
    labels = torch.tensor(labels, dtype=torch.long)

    # Token hóa văn bản
    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,  # Thêm token đặc biệt [CLS], [SEP]
        truncation=True,          # Cắt chuỗi dài hơn `max_length`
        padding=True,             # Thêm padding để các chuỗi có cùng độ dài
        max_length=max_seq_length,
        return_tensors="pt"       # Trả về tensor PyTorch
    )

    # Kiểm tra kích thước tensor đầu ra từ tokenizer
    print("\n===== Tokenized Inputs Shape =====")
    print("Input IDs Shape:", token_inputs["input_ids"].shape)
    print("Attention Mask Shape:", token_inputs["attention_mask"].shape)

    # Thêm nhãn vào batch
    token_inputs.update({"labels": labels})

    return token_inputs


`tokenizer` xuất hiện trong hàm `collate_fn` vì nó đã được khởi tạo trước đó bằng cách sử dụng hàm `initialize_model_and_tokenizer`. Khi bạn khởi tạo `tokenizer` ở đây:

```python
tokenizer, model, max_seq_length = initialize_model_and_tokenizer(
    model_class=BERTIntentClassification,
    model_name="bert-base-uncased",
    num_classes=5,
    max_seq_length=512,
    cache_dir="huggingface"
)
```

`tokenizer` trở thành một đối tượng toàn cục (global) trong phạm vi của file script hoặc notebook mà bạn đang làm việc. Vì vậy, nó có thể được sử dụng trực tiếp bên trong `collate_fn`.

---

### **Tại sao `tokenizer` cần được sử dụng trong `collate_fn`?**

Hàm `collate_fn` được dùng trong quá trình huấn luyện hoặc đánh giá mô hình. Nhiệm vụ chính của `collate_fn` là chuẩn bị dữ liệu đầu vào cho mô hình. `tokenizer` trong `collate_fn` có nhiệm vụ chuyển đổi danh sách văn bản (`inputs`) sang định dạng số hóa mà mô hình có thể hiểu.

Cụ thể:
1. **Token hóa văn bản (`inputs`)**:
   - Chuyển từ danh sách các chuỗi văn bản (`inputs`) thành:
     - `input_ids`: Các mã số (IDs) đại diện cho các token.
     - `attention_mask`: Đánh dấu token nào cần được mô hình chú ý.

2. **Padding và Truncation**:
   - Đảm bảo tất cả các câu có độ dài cố định (`max_seq_length`) bằng cách thêm các token đệm (padding) hoặc cắt bớt (truncation).

3. **Trả về Tensor**:
   - Kết quả được token hóa sẽ trả về ở dạng tensor để tương thích với PyTorch.



In [22]:
train_dataset
print("===== Train Dataset =====")
for sample in train_dataset:
    print(sample)


===== Train Dataset =====
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}
{'text': 'Không, tôi không thấy cần thiết.', 'label': 3}
{'text': 'Chắc chắn rồi, để tôi làm.', 'label': 4}
{'text': 'Không, tôi không đồng ý.', 'label': 3}
{'text': 'Hoàn toàn chính xác, để tôi làm.', 'label': 4}
{'text': 'Im lặng trong 10 giây.', 'label': 1}
{'text': 'Không, tôi nghĩ là không cần.', 'label': 3}
{'text': 'Không có gì để nói cả.', 'label': 1}
{'text': 'EMPTY_TEXT', 'label': 1}
{'text': 'Vâng, tôi muốn cho bạn xem bức tranh.', 'label': 4}
{'text': 'Vâng, tôi đồng ý với bạn.', 'label': 4}
{'text': 'Xin lỗi, tôi không chắc bạn đang nói gì.', 'label': 0}
{'text': 'Tôi nghĩ tôi cần thêm thời gian để xem xét.', 'label': 2}
{'text': 'Không, điều đó không đúng.', 'label': 3}
{'text': 'Không có phản hồi nào trong 5 giây.', 'label': 1}
{'text': 'Không có âm thanh nào phát ra.', 'label': 1}
{'text': 'Tôi không biết.', 'label': 2}
{'text': 'Xin lỗi, tôi không hiểu bạn đang nói gì.', 'label': 0}
{'text': '

In [51]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=8,
    collate_fn=collate_fn
)

In [52]:
for batch in train_dataloader:
    print("\n===== Batch from DataLoader =====")
    print(batch)
    break  # Kiểm tra 1 batch đầu tiên



===== Features in Batch (First Sample) =====
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}

===== Batch Size: 8 =====
Sample Text: Bạn có thể nhắc lại không?
Sample Label: 0

===== Tokenized Inputs Shape =====
Input IDs Shape: torch.Size([8, 17])
Attention Mask Shape: torch.Size([8, 17])

===== Batch from DataLoader =====
{'input_ids': tensor([[  101,  7221,  2522,  1996, 18699,  6305, 21110,  1047, 19991,  1029,
           102,     0,     0,     0,     0,     0,     0],
        [  101,  1047, 19991,  1010,  2000,  2072,  1047, 19991, 22794,  2100,
          2064, 16215,  2666,  2102,  1012,   102,     0],
        [  101, 15775,  2278,  9212, 25223,  1010,  1102,  2063,  2000,  2072,
         16983,  1012,   102,     0,     0,     0,     0],
        [  101,  1047, 19991,  1010,  2000,  2072,  1047, 19991,  1102,  5063,
          1061,  1012,   102,     0,     0,     0,     0],
        [  101,  7570,  2319,  2000,  2319,  5413,  2232,  1060,  6305,  1010,
          1102,  2063,  2

Output trên là batch dữ liệu sau khi được xử lý bởi `collate_fn` và chuẩn bị sẵn sàng để đưa vào mô hình BERT. Dưới đây là giải thích ngắn gọn cho từng phần:

---

### **1. `Features in Batch (First Sample)`**
```plaintext
{'text': 'Vâng, tôi muốn cho bạn xem bức tranh.', 'label': 4}
```
- Đây là một mẫu đầu tiên trong batch, chứa:
  - **`text`**: Chuỗi văn bản gốc.
  - **`label`**: Nhãn dạng số nguyên, ở đây là `4` (đã chuyển đổi từ nhãn chuỗi).

---

### **2. `Batch Size` và `Sample Text/Label`**
```plaintext
Batch Size: 8
Sample Text: Vâng, tôi muốn cho bạn xem bức tranh.
Sample Label: 4
```
- **Batch Size**: Số lượng mẫu trong batch này là `8`.
- **Sample Text/Label**: Văn bản và nhãn của mẫu đầu tiên trong batch.

---

### **3. `Tokenized Inputs Shape`**
```plaintext
Input IDs Shape: torch.Size([8, 20])
Attention Mask Shape: torch.Size([8, 20])
```
- **`Input IDs`**: Tensor chứa token IDs của batch, kích thước `[8, 20]`, tương ứng với:
  - `8`: Số mẫu trong batch.
  - `20`: Độ dài tối đa của các chuỗi sau khi padding.
- **`Attention Mask`**: Tensor đánh dấu các vị trí thật (`1`) và padding (`0`), cùng kích thước với `Input IDs`.

---

### **4. `Batch from DataLoader`**
```plaintext
{'input_ids': ..., 'attention_mask': ..., 'labels': ...}
```
- **`input_ids`**: Token IDs của từng câu sau khi tokenization.
  - Padding được thêm vào để đảm bảo tất cả các câu có độ dài bằng `20`.
- **`attention_mask`**: Đánh dấu:
  - `1`: Token thật (từ chuỗi gốc).
  - `0`: Token padding.
- **`labels`**: Tensor chứa nhãn của batch:
  ```plaintext
  tensor([4, 2, 1, 2, 0, 1, 3, 0])
  ```

---

### **Tóm lại**
- Batch đã được chuẩn bị đầy đủ:
  - Token hóa và padding để phù hợp với BERT.
  - Nhãn ở dạng số nguyên.
- Đây là đầu vào sẵn sàng cho mô hình trong quá trình huấn luyện.

Login Wandb

In [38]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.1
    Uninstalling wandb-0.19.1:
      Successfully uninstalled wandb-0.19.1


In [26]:
!pip install python-dotenv

In [39]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [40]:
# In giá trị của API key
wandb_api_key = os.getenv("WANDB_API_KEY")
if wandb_api_key:
    print(f"API Key Loaded: {wandb_api_key[:5]}...")  # Chỉ in 5 ký tự đầu để kiểm tra
else:
    print("API Key not found in environment variables.")


API Key Loaded: c8767...


In [43]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Cài tham số models

### **Kiến trúc code**

1. **Mục đích chính**:
   - Đoạn code thiết kế một mô hình phân loại ý định (`Intent Classification`) dựa trên BERT và tùy chỉnh huấn luyện với Hugging Face `Trainer`.

2. **Thành phần chính**:
   - **Lớp `BERTIntentClassification`**:
     - Kế thừa từ `torch.nn.Module`.
     - Sử dụng BERT (`AutoModel`) làm backbone để trích xuất đặc trưng văn bản.
     - Sử dụng một mạng `Feed-Forward Neural Network (FFNN)` làm tầng phân loại với dropout và ReLU.
     - Triển khai phương pháp pooling (mean pooling) từ `hidden_state` của BERT với `attention_mask`.

   - **Phương thức `freeze_bert`**:
     - Đóng băng các tham số của BERT, chỉ huấn luyện tầng phân loại FFNN để giảm chi phí tính toán.

   - **Lớp `TrainerCustom`**:
     - Tùy chỉnh lớp `Trainer` của Hugging Face để định nghĩa cách tính loss (`compute_loss`) sử dụng `nn.CrossEntropyLoss`.

3. **Cách hoạt động**:
   - Tải mô hình BERT từ Hugging Face.
   - Truyền dữ liệu qua BERT để lấy đặc trưng (hidden states).
   - Sử dụng mean pooling để tạo biểu diễn cho chuỗi đầu vào.
   - Truyền qua FFNN để dự đoán nhãn (logits).
   - Huấn luyện và tính loss với `TrainerCustom`.

4. **Mở rộng**:
   - Hỗ trợ tùy chỉnh số nhãn (`num_classes`), dropout, và các cài đặt khác.
   - Sử dụng mean pooling, có thể mở rộng sang max pooling hoặc CLS token.

In [48]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm

# Thiết lập sử dụng GPU cụ thể (ở đây là GPU 1)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Setup CUDA GPU 1

# Định nghĩa class mô hình phân loại ý định sử dụng BERT
class BERTIntentClassification(nn.Module):

    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir="huggingface"):
        """
        Khởi tạo mô hình phân loại ý định sử dụng BERT.
        Args:
            model_name: Tên mô hình BERT được tải từ Hugging Face.
            num_classes: Số lớp (nhãn) cần phân loại.
            dropout_rate: Tỷ lệ dropout trong FFNN để tránh overfitting.
            cache_dir: Thư mục lưu cache mô hình BERT.
        """
        super(BERTIntentClassification, self).__init__()
        # Tải mô hình BERT từ Hugging Face
        self.bert = AutoModel.from_pretrained(model_name, cache_dir=cache_dir)
        # Lấy kích thước hidden của BERT
        hidden_size = self.bert.config.hidden_size
        # Định nghĩa mạng FFNN cho đầu ra
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),  # Lớp fully connected
            nn.LayerNorm(hidden_size),  # Chuẩn hóa layer
            nn.ReLU(),  # Hàm kích hoạt ReLU
            nn.Dropout(dropout_rate),  # Dropout
            nn.Linear(hidden_size, num_classes)  # Lớp fully connected cho đầu ra
        )

    def freeze_bert(self):
        """
        Đóng băng các tham số của BERT, chỉ cập nhật tham số của lớp FFNN.
        """
        for param in self.bert.parameters():
            param.requires_grad = False

    def get_pooling(self, hidden_state, attention_mask):
        """
        Hàm trích xuất biểu diễn (representation) đã được mean pooling từ hidden states của BERT.
        Args:
            hidden_state: Hidden states đầu ra từ BERT.
            attention_mask: Attention mask để xác định các token có giá trị thực.
        Returns:
            pooled_output: Biểu diễn trung bình của toàn bộ chuỗi đầu vào.
        """
        # Lấy last hidden state từ đầu ra của BERT
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Mở rộng attention mask để phù hợp với kích thước hidden state
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask các token padding
            masked_hidden = last_hidden_state * attention_mask

            # Tính tổng các hidden states
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            # Đếm số lượng token thực sự (không phải padding)
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            # Tính trung bình (tổng chia cho số token)
            pooled_output = sum_hidden / count_tokens
        else:
            # Nếu không có attention mask, lấy trung bình tất cả các token
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output

    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass của mô hình.
        Args:
            input_ids: IDs của các token đầu vào.
            attention_mask: Attention mask để xác định token padding.
        Returns:
            logits: Đầu ra raw logits cho mỗi lớp.
        """
        # Lấy hidden states từ BERT
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Trích xuất biểu diễn đã được pooling
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Truyền qua lớp FFNN để tạo logits
        logits = self.ffnn(hidden_state_pooling)

        return logits


# Custom Trainer class để điều chỉnh cách tính loss
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Hàm tính toán loss được sử dụng bởi Trainer.
        Args:
            model: Mô hình đang huấn luyện.
            inputs: Dữ liệu đầu vào, bao gồm input_ids, attention_mask, và labels.
            return_outputs: Có trả về outputs hay không.
        Returns:
            loss: Giá trị loss đã tính toán.
        """
        # Tách labels từ inputs (nếu có)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() để tính toán loss
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và lấy đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo đầu ra là logits
        logits = outputs

        # Tính toán loss dựa trên logits và labels
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


In [49]:
# # Bước 6: Cài đặt tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="./results",          # Thư mục lưu kết quả
#     eval_strategy="epoch",    # Đánh giá sau mỗi epoch
#     learning_rate=2e-4,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     num_train_epochs=50,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=None,
#     logging_strategy = "epoch",
#     save_strategy="epoch",          # Lưu trọng số sau mỗi epoch
#     save_total_limit=3,
# )

# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,  # Ghi log sau mỗi 10 bước
    logging_strategy="steps"  # Đảm bảo ghi log theo steps
)




In [62]:
print("\n===== Train Dataset =====")
for sample in sample_train:
    print(sample)

print("\n===== Eval Dataset =====")
for sample in sample_test:
    print(sample)

for batch in train_dataloader:
    print("\n===== Batch from DataLoader =====")
    print(batch)
    break

def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)
    # Đảm bảo không xóa "text"
    return example




===== Train Dataset =====
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}
{'text': 'Không, tôi không thấy cần thiết.', 'label': 3}
{'text': 'Chắc chắn rồi, để tôi làm.', 'label': 4}
{'text': 'Không, tôi không đồng ý.', 'label': 3}
{'text': 'Hoàn toàn chính xác, để tôi làm.', 'label': 4}

===== Eval Dataset =====
{'text': 'Tôi không chắc điều này có cần thiết không.', 'label': 2}
{'text': 'Tôi không chắc, hãy để tôi kiểm tra lại.', 'label': 2}

===== Features in Batch (First Sample) =====
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}

===== Batch Size: 8 =====
Sample Text: Bạn có thể nhắc lại không?
Sample Label: 0

===== Tokenized Inputs Shape =====
Input IDs Shape: torch.Size([8, 17])
Attention Mask Shape: torch.Size([8, 17])

===== Batch from DataLoader =====
{'input_ids': tensor([[  101,  7221,  2522,  1996, 18699,  6305, 21110,  1047, 19991,  1029,
           102,     0,     0,     0,     0,     0,     0],
        [  101,  1047, 19991,  1010,  2000,  2072,  1047, 19991, 22

In [59]:
def collate_fn(features):
    """
    Chuẩn bị batch dữ liệu để huấn luyện mô hình BERT.
    """
    # In thông tin từng phần tử trong batch
    for feature in features:
        print(f"Feature: {feature}")
        if "text" not in feature:
            print(f"Missing key 'text' in feature: {feature}")
        if "label" not in feature:
            print(f"Missing key 'label' in feature: {feature}")

    inputs = [feature["text"] for feature in features if "text" in feature]
    labels = [feature["label"] for feature in features if "label" in feature]

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )

    token_inputs.update({"labels": labels})
    return token_inputs


In [60]:
# Bước 7: Tạo Trainer
trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=sample_train,
    eval_dataset=sample_test,
    tokenizer=tokenizer,
    data_collator = collate_fn,
)

# Bước 8: Huấn luyện
trainer.train()

<ipython-input-60-ba8ccdf04d93>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerCustom.__init__`. Use `processing_class` instead.
  trainer = TrainerCustom(


Feature: {'label': 4}
Missing key 'text' in feature: {'label': 4}
Feature: {'label': 4}
Missing key 'text' in feature: {'label': 4}
Feature: {'label': 3}
Missing key 'text' in feature: {'label': 3}
Feature: {'label': 0}
Missing key 'text' in feature: {'label': 0}
Feature: {'label': 3}
Missing key 'text' in feature: {'label': 3}


IndexError: list index out of range

In [ ]:
# Bước 9: Đánh giá trên tập kiểm tra
trainer.evaluate()